# Remove Marker in fMOST Brain (Definitive Edition)

Algorithm:
1. Closing (Smoothing)
2. Sobel
3. Canny
4. Closing (Filling holes)
5. Hough Lines Probability Transform
6. Filter lines based on orientation & distance to center
7. Draw lines (extending length & width)
8. Closing (Filling gaps over z direction)

## Functions

In [1]:
import cv2
import numpy as np
import SimpleITK as sitk
import ipywidgets as widgets
import os

### Functions for Loading data from teraconvert

In [2]:
def load_entire_resolution(dir):
    '''
    :param dir: path to a resolution
    :return: assembled image stack (by z, y, x) of that resolution
    '''

    # _folders: all the folder in the corresponding dir of that dimension
    # _start: the starting coord of the current dimension of the iterated block
    # z_files: tiff image blocks

    y_folders = [d for d in os.listdir(dir) if os.path.isdir(os.path.join(dir, d))]
    ensemble_array = []
    for i, y_folder in enumerate(y_folders):
        y_dir = os.path.join(dir, y_folder)
        x_folders = [d for d in os.listdir(y_dir) if os.path.isdir(os.path.join(y_dir, d))]
        x_array = []
        for j, x_folder in enumerate(x_folders):
            x_dir = os.path.join(y_dir, x_folder)
            z_files = [d for d in os.listdir(x_dir)]
            z_array = []
            for k, z_file in enumerate(z_files):
                z_path = os.path.join(x_dir, z_file)
                img = sitk.ReadImage(z_path)
                arr = sitk.GetArrayFromImage(img).transpose([1, 2, 0])
                z_array.append(arr)
            x_array.append(z_array)
        ensemble_array.append(x_array)
    return np.block(ensemble_array)


def load_from_teraconvert(path_to_brain_id):
    '''
    :param path_to_brain_id: the path to the teraconvert dir of a brain
    :return: the lowest resolution brain (as numpy array) of that dir, by z, y, x
    '''
    res = os.listdir(path_to_brain_id)
    res_x = [str(name.split('x')[1]) for name in res]
    min_res = res[np.argmin(res_x)]
    min_res_dir = os.path.join(path_to_brain_id, min_res)

    return load_entire_resolution(min_res_dir).transpose([2, 0, 1])

### Generate Marker Mask

In [48]:
def remove_marker(img, **kwargs):
    '''
    :param img: 3d stack numpy array (z,y,x)
    :return: the binary mask of the marker (255 for marker)
    '''

    stack = []
    SE1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kwargs['SE1'],)*2)
    SE2 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kwargs['SE2'],)*2)
    SE3 = cv2.getStructuringElement(cv2.MORPH_RECT, (1, kwargs['SE3']))
    gk = kwargs['sigma'] * 3
    if gk % 2 == 0:
        gk += 1
    for sl in img:
        # Closing
        closed = cv2.morphologyEx(sl, cv2.MORPH_CLOSE, SE1)
        closed = cv2.GaussianBlur(closed, (gk,) * 2, kwargs['sigma'])
        # Canny
        dx = cv2.Sobel(closed, cv2.CV_16SC1, 1, 0)
        dy = cv2.Sobel(closed, cv2.CV_16SC1, 0, 1)
        m = np.hypot(dx, dy).max()
        canny = cv2.Canny(dx, dy, kwargs['cannyMin'] * m, kwargs['cannyMax'] * m)
        stack.append(canny)
    stack = np.stack(stack)

    # get all marker lines
    all_lines = []
    for i, sl in enumerate(stack):
        # Closing
        closed = cv2.morphologyEx(sl, cv2.MORPH_CLOSE, SE2)
        # Hough Line Transform P
        lines = cv2.HoughLinesP(closed, kwargs['rho'], kwargs['theta'],
                                kwargs['threshold'], minLineLength=kwargs['minLineLength'], maxLineGap=kwargs['maxLineGap'])
        if lines is not None:
            # reshape lines so that it looks like:
            # [ [[x1, y1], [x2, y2]], .. ]
            # and append the slice info so that it looks like
            # [ [[x1, y1, z1], [x2, y2, z2]], .. ]
            lines = np.append(lines.reshape(-1, 2, 2), np.ones([len(lines), 2, 1], dtype=np.int32) * i, axis=2)
            all_lines.extend(lines)

    # filter lines based on distance to center and orientation
    fct = [1, 1, kwargs['zThickness']]
    ct = np.flip(img.shape) * fct / 2
    def pass_dist(p1, p2):
        _p1 = p1 * fct
        _p2 = p2 * fct
        return np.linalg.norm(np.cross(_p2 - ct, _p1 - ct)) / np.linalg.norm(_p2 - _p1) >= kwargs['minDist']

    def pass_angle(p1, p2):
        def get_angle(v1, v2):
            inner = np.inner(v1, v2)
            norms = np.linalg.norm(v1) * np.linalg.norm(v2)
            cos = abs(inner / norms)
            rad = np.arccos(np.clip(cos, -1.0, 1.0))
            deg = np.rad2deg(rad)
            return abs(deg)
        if ct[0] > ct[1]:
            return get_angle(p1 - p2, np.array([0, 1, 0])) <= kwargs['angleLim']
        else:
            return get_angle(p1 - p2, np.array([1, 0, 0])) <= kwargs['angleLim']

    filtered_lines = [line for line in all_lines if pass_angle(*line) and pass_dist(*line)]

    # draw the mask
    stack = np.zeros(stack.shape, dtype=np.uint8)
    for p1, p2 in filtered_lines:
        _p1 = p1[:2]
        _p2 = p2[:2]
        d = _p2 - _p1
        _p2 = _p2 + d * kwargs['extendRatio']
        _p1 = _p1 - d * kwargs['extendRatio']
        cv2.line(stack[p1[2]], _p1.astype('int'), _p2.astype('int'), 255, kwargs['lineWidth'])

    # cross z closing
    stack = stack.reshape(stack.shape[0], -1)
    stack = cv2.morphologyEx(stack, cv2.MORPH_CLOSE, SE3)
    stack = stack.reshape(img.shape)

    return stack

## Run

In [11]:
in_dir = '../results/lowestRes'
out_dir = '../results/removeband'
for brain in os.listdir(in_dir):
    ip = os.path.join(in_dir, brain)
    op = os.path.join(out_dir, brain)
    img = sitk.ReadImage(ip)
    img = sitk.GetArrayFromImage(img)
    out = remove_marker(img, SE1=11, SE2=5, lineWidth=3, SE3=21, extendRatio=0.1, cannyMin=0.1, cannyMax=0.3, sigma=1,
                        # hough
                        rho=1, theta=np.pi / 180, threshold=100, minLineLength=100, maxLineGap=1,
                        # filter
                        minDist=300, angleLim=5, zThickness=2)
    sitk_io = sitk.GetImageFromArray(out)
    sitk.WriteImage(sitk_io, op)
    print(brain, 'done.')

17302.tif done.
17544.tif done.
18049.tif done.
182724.tif done.
182726.tif done.
182727.tif done.
18454.tif done.
18455.tif done.
18463.tif done.
191807.tif done.
192333.tif done.
192334.tif done.
192342.tif done.
192346.tif done.
192348.tif done.
194060.tif done.
194063.tif done.
194066.tif done.
194069.tif done.
196469.tif done.


In [55]:
no = '182724'
img = sitk.ReadImage('../results/lowestRes/'+ no +'.tif')
img = sitk.GetArrayFromImage(img)
out = remove_marker(img, SE1=11, SE2=5, lineWidth=3, SE3=21, extendRatio=0.1, cannyMin=0.1, cannyMax=0.3, sigma=1,
                        # hough
                        rho=1, theta=np.pi / 180, threshold=100, minLineLength=100, maxLineGap=1,
                        # filter
                        minDist=300, angleLim=5, zThickness=2)
sitk_io = sitk.GetImageFromArray(out)
sitk.WriteImage(sitk_io, '../results/removeband/' + no + '.tif')